In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, to_date, regexp_replace, lower, when

spark = SparkSession.builder.appName("ProcesamientoTweets").getOrCreate()

df = spark.read.csv("bigdata-sentiment/data/tweets.csv", header=True, inferSchema=True)

df_clean = df.withColumn("text_clean", lower(col("text")))
df_clean = df_clean.withColumn("text_clean", regexp_replace("text_clean", r"http\S+", ""))
df_clean = df_clean.withColumn("text_clean", regexp_replace("text_clean", r"@\w+", ""))
df_clean = df_clean.withColumn("text_clean", regexp_replace("text_clean", r"[^\w\s]", ""))
df_clean = df_clean.withColumn("text_clean", regexp_replace("text_clean", r"\s+", " "))

df_clean = df_clean.withColumn("tokens", regexp_replace(col("text_clean"), " ", ","))

df_clean = df_clean.withColumn("sentiment_numeric",
    when(col("sentiment") == "positive", 1)
    .when(col("sentiment") == "negative", -1)
    .otherwise(0)
)

df_clean = df_clean.withColumn("date_formatted", to_date(col("date"), "yyyy-MM-dd"))
df_clean = df_clean.withColumn("text_length", length(col("text_clean")))

df_clean.show(10, truncate=False)